In [2]:
# notebook init
# Ensure that you have cloned the road-network-inference repo into /content/drive/MyDrive/
# Make sure the notebook is running using a GPU
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet
!pip install PyMaxflow
import os
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V
from tqdm import tqdm 
from google.colab.patches import cv2_imshow
from skimage.draw import line
import cv2
import os
import math
import numpy as np
import pandas as pd

from time import time
from networks.dinknet import ResNet34_EdgeNet



%cd /content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet/wrapper/bilateralfilter
%run /content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet/wrapper/bilateralfilter/setup.py build
%run /content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet/wrapper/bilateralfilter/setup.py install
%cd /content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet
/content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor/DBNet/wrapper/bilateralfilter
running build
running build_py
running build_ext
running install
running build
running build_py
running build_ext
running install_lib
running install_egg_info
Removing /usr/local/lib/python3.7/dist-packages/bilateralfilter-0.1.egg-info
Writing /usr/local/lib/python3.7/dist-packages/bilateralfilter-0.1.egg-info
/content/drive/MyDrive/Road-Network-Inference/ScRoadExtractor


In [3]:
# Path to the saved model weights
weights = './weights/DBNet_10Cities_zoomed_2.th'
# Path to the data csv file which is generated by the dataset creation
csv_dir = '/content/drive/MyDrive/Road-Network-Inference/data_backup.csv'
# path to the output csv. We highly recommend keeping this the same for road width and lane detection so that the outputs end up in the same file
output_csv = '/content/drive/MyDrive/Road-Network-Inference/samples_out.csv'

In [4]:
# Measures the road width based on the trained model and outputs it to a csv

BATCHSIZE_PER_CARD = 2

class TTAFrame():
    def __init__(self, net):
        self.net = net().cuda()
        self.net = torch.nn.DataParallel(self.net, device_ids=range(torch.cuda.device_count()))
        
    def test_one_img_from_path(self, path, evalmode=True):
        if evalmode:
            self.net.eval()
        batchsize = torch.cuda.device_count() * BATCHSIZE_PER_CARD
        if batchsize >= 8:
            return self.test_one_img_from_path_1(path)
        elif batchsize >= 4:
            return self.test_one_img_from_path_2(path)
        elif batchsize >= 2:
            return self.test_one_img_from_path_4(path)

    def test_one_img_from_path_8(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]

        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)

        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())

        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()

        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]

        return mask2

    def test_one_img_from_path_4(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None], img90[None]])
        img2 = np.array(img1)[:, ::-1]
        img3 = np.array(img1)[:, :, ::-1]
        img4 = np.array(img2)[:, :, ::-1]

        img1 = img1.transpose(0, 3, 1, 2)
        img2 = img2.transpose(0, 3, 1, 2)
        img3 = img3.transpose(0, 3, 1, 2)
        img4 = img4.transpose(0, 3, 1, 2)

        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())

        maska, edgea = self.net.forward(img1)
        maskb, edgeb = self.net.forward(img2)
        maskc, edgec = self.net.forward(img3)
        maskd, edged = self.net.forward(img4)

        maska = maska.squeeze().cpu().data.numpy()
        maskb = maskb.squeeze().cpu().data.numpy()
        maskc = maskc.squeeze().cpu().data.numpy()
        maskd = maskd.squeeze().cpu().data.numpy()

        edgea = edgea.squeeze().cpu().data.numpy()
        edgeb = edgeb.squeeze().cpu().data.numpy()
        edgec = edgec.squeeze().cpu().data.numpy()
        edged = edged.squeeze().cpu().data.numpy()

        mask1 = maska + maskb[:, ::-1] + maskc[:, :, ::-1] + maskd[:, ::-1, ::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1, ::-1]

        edge1 = edgea + edgeb[:, ::-1] + edgec[:, :, ::-1] + edged[:, ::-1, ::-1]
        edge2 = edge1[0] + np.rot90(edge1[1])[::-1, ::-1]

        return mask2, edge2
    
    def test_one_img_from_path_2(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None], img90[None]])
        img2 = np.array(img1)[:, ::-1]
        img3 = np.concatenate([img1, img2])
        img4 = np.array(img3)[:, :, ::-1]
        img5 = img3.transpose(0, 3, 1, 2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 - 1.6
        img5 = V(torch.Tensor(img5).cuda())
        img6 = img4.transpose(0, 3, 1, 2)
        img6 = np.array(img6, np.float32)/255.0 * 3.2 - 1.6
        img6 = V(torch.Tensor(img6).cuda())
        
        maska = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        maskb = self.net.forward(img6).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3
    
    def test_one_img_from_path_1(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None], img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = np.concatenate([img3,img4]).transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        
        mask = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        mask1 = mask[:4] + mask[4:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3

    def load(self, path):
        self.net.load_state_dict(torch.load(path))

maindir = '/content/drive/MyDrive/Road-Network-Inference/'

if os.path.exists(output_csv):
  csv_dir = output_csv
df = pd.read_csv(csv_dir)

solver = TTAFrame(ResNet34_EdgeNet)

solver.load(weights)
tic = time()

output = []
output.append(df.columns.values.tolist() + ["predicted_road_width_meters"])
for i, row in df.iterrows():
    
  sat_name = row['sat_name']
  city_name = row['city_name']
  lat = row['n1_lat']
  gsd = 156543.03392 * math.cos(lat * math.pi / 180) / math.pow(2, 20) 
  x1 = int(row['perp1_x'])
  y1 = int(row['perp1_y'])
  x2 = int(row['perp2_x'])
  y2 = int(row['perp2_y'])
  
  
  midpoint = (int((x1+x2)/2), int((y1+y2)/2))
  rr, cc = line(midpoint[0],midpoint[1],x2,y2)
  points1 = list(zip(rr,cc))
  rr, cc = line(midpoint[0],midpoint[1],x1,y1)
  points2 = list(zip(rr,cc))
  mask, edge = solver.test_one_img_from_path(maindir + sat_name)

  mask_binary = mask.copy()
  mask_binary[mask_binary > 4] = 255
  mask_binary[mask_binary <= 4] = 0
  for y, x in points1:
    if (mask_binary[x][y] ==0):
      x1 = x
      y1 = y
      break
  for y, x in points2:
  
    if (mask_binary[x][y] ==0):
      x2 = x
      y2 = y
      break
  dist = ((x2-x1)**2 + (y2-y1)**2)**.5
  print
  output.append(row.values.flatten().tolist() + [gsd*dist])

np.savetxt(output_csv, output, delimiter=',', fmt='%s')







Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

4
4
4
